In [1]:
import tensorflow as tf

In [2]:
# Set tensorflow config
tf.set_random_seed(1)
tf.logging.set_verbosity(tf.logging.DEBUG)

In [8]:
# Define flags 

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string(
    flag_name='model_dir', 
    default_value='./estimator_model', 
    docstring='Output directory for model and training stats.'
)

In [9]:
def run_experiment(config):
    print(co)

In [10]:
if __name__ == "__main__":
    tf.app.run(
        main=run_experiment
    )

Hello


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
